## Mini Project III

### Topics
This mini project is dedicated to following topics:
- Data Wrangling
- Data Visualization
- Data Preparation and Feature Engineering
- Dimensionality Reduction
- Unsupervised Learning

### Data
We will be using old data about different financial transactions. You can download the data from [here](https://drive.google.com/file/d/1zAjnf936aHkwVCq_BmA47p4lpRjyRzMf/view?usp=sharing). The data contains following tables:

- twm_customer - information about customers
- twm_accounts - information about accounts
- twm_checking_accounts - information about checking accounts (subset of twm_accounts)
- twm_credit_accounts - information about checking accounts (subset of twm_accounts)
- twm_savings_accounts - information about checking accounts (subset of twm_accounts)
- twm_transactions - information about financial transactions
- twm_savings_tran - information about savings transactions (subset of twm_transactions)
- twm_checking_tran - information about savings transactions (subset of twm_transactions)
- twm_credit_tran - information about credit checking (subset of twm_transactions)


### Output

In this miniproject, we will:

1.  create two separate customer segmentations (using clustering) to split them into 3-5 clusters: 
    - based on demographics (only on the information from twm_customer)
    - based on their banking behavior. We can take following things into consideration as banking behavior:
        - do they have savings account? How much do they save?
        - do they have credit account? How much do they live in debt?
        - are they making lot of small transactions or few huge ones?
2. visualize the created clusters using [radar charts](https://plotly.com/python/radar-chart/) and compare them agains each other
3. visualize segmentations using scatter plot. We will have to use PCA to be able to plot our observations in 2D.
4. (stretch) visualize in 2D how our clusters are evolving in each iteration of KMeans (for at least 20 iterations).
    - we will need to create own implementation of kmeans so we can see what is happening with the clusters during the iterations.


## Project Plan

### Data Preparation 
- Data Exploration:
    - Explore data
    - Explore the target variable (if we have one)
- Find relationships between target and other variables
- Data cleaning
    - Missing values
    - Outlier detection

- Creating a db diagram 
- load data
- idenitfy columns will nulls (remove or replace)
- find data types
- change date columns to datetime
- change categories into 1/2
- create db

In [87]:
import pandas as pd
import numpy as np
import requests
from pandas import json_normalize
from IPython.display import JSON
import json
import sqlite3 as sqlite
from sqlalchemy import create_engine

### df_accounts

In [24]:
df_accounts = pd.read_csv('twm_accounts.csv',";")
print(df_accounts.shape)
df_accounts.head(5)

(1409, 8)


acct_nbr  cust_id acct_type account_active acct_start_date  \
0          13628063  1362806      SV               Y       10.12.1995   
1  4561143213627090  1362709      CC               Y        15.3.1993   
2  4561143213628360  1362836      CC               Y        18.3.1992   
3          13633112  1363311      CK               Y         6.7.1995   
4  4561143213633610  1363361      CC               Y        17.6.1994   

  acct_end_date  starting_balance  ending_balance  
0           NaN           1430.22          284.58  
1           NaN            266.34          496.15  
2           NaN             55.90         1000.00  
3           NaN          11017.13          968.46  
4           NaN            849.37          462.28

In [31]:
df_accounts2 = df_accounts.copy()

In [32]:
total = df_accounts2.isnull().sum().sort_values(ascending=False)
percent = (df_accounts2.isnull().sum()/df_accounts.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

Total  Percent
acct_end_date      1266  0.89851
ending_balance        0  0.00000
starting_balance      0  0.00000
acct_start_date       0  0.00000
account_active        0  0.00000
acct_type             0  0.00000
cust_id               0  0.00000
acct_nbr              0  0.00000

In [33]:
to_drop = missing_data.head(1).index.tolist()
df_accounts2.drop(to_drop, axis=1, inplace=True)

In [34]:
df_accounts2.head()

acct_nbr  cust_id acct_type account_active acct_start_date  \
0          13628063  1362806      SV               Y       10.12.1995   
1  4561143213627090  1362709      CC               Y        15.3.1993   
2  4561143213628360  1362836      CC               Y        18.3.1992   
3          13633112  1363311      CK               Y         6.7.1995   
4  4561143213633610  1363361      CC               Y        17.6.1994   

   starting_balance  ending_balance  
0           1430.22          284.58  
1            266.34          496.15  
2             55.90         1000.00  
3          11017.13          968.46  
4            849.37          462.28

In [36]:
df_accounts2.to_csv('df_accounts_cleaned.csv', sep=',', index=False)

### df_checking_acct

In [4]:
df_checking_acct = pd.read_csv('twm_checking_acct.csv',";")
print(df_checking_acct.shape)
df_checking_acct.head(5)

(520, 9)


cust_id  acct_nbr  minimum_balance  per_check_fee account_active  \
0  1362548  13625482             3000           0.00             Y    
1  1362487  13624872             3000           0.00             Y    
2  1363017  13630172             3000           0.00             Y    
3  1362752  13627522              100           0.15             Y    
4  1363282  13632822              100           0.15             Y    

  acct_start_date acct_end_date  starting_balance  ending_balance  
0      11.11.1986           NaN           6004.34          569.65  
1       12.1.1995           NaN           2781.07         1401.21  
2       19.3.1994           NaN           2694.91          147.15  
3        8.1.1994           NaN            487.69            3.76  
4        5.3.1988           NaN            133.90           84.18

In [104]:
df_checking_acct2 = pd.read_csv('df_checking_acct_clean.csv',',')
df_checking_acct2 = df_checking_acct2.drop(labels='Unnamed: 0', axis=1)
df_checking_acct2.head()

cust_id  acct_nbr  minimum_balance  per_check_fee account_active  \
0  1362548  13625482             3000           0.00             Y    
1  1362487  13624872             3000           0.00             Y    
2  1363017  13630172             3000           0.00             Y    
3  1362752  13627522              100           0.15             Y    
4  1363282  13632822              100           0.15             Y    

  acct_start_date  starting_balance  ending_balance  
0      11.11.1986           6004.34          569.65  
1       12.1.1995           2781.07         1401.21  
2       19.3.1994           2694.91          147.15  
3        8.1.1994            487.69            3.76  
4        5.3.1988            133.90           84.18

### df_checking_tran

In [115]:
df_checking_tran = pd.read_csv('twm_checking_tran.csv',";")
print(df_checking_tran.shape)
df_checking_tran.head(30)

(46204, 10)


cust_id  tran_id  tran_amt  principal_amt  interest_amt  new_balance  \
0   1363481       26     -0.15          -0.15          0.00        58.99   
1   1362784       90   -200.00        -200.00          0.00      1380.69   
2   1363088       46      0.00           0.00          0.00       128.35   
3   1363306       32    -40.32         -40.32          0.00       433.06   
4   1363251       11    -87.88         -87.88          0.00      1919.55   
5   1363303       20    -58.58         -58.58          0.00      2660.59   
6   1362981       39   -311.06        -311.06          0.00       564.99   
7   1363216       97    -45.11         -45.11          0.00       480.60   
8   1363253       62      0.94           0.00          0.94       751.88   
9   1363442       16     -0.75          -0.75          0.00      4776.26   
10  1362959       10   -258.17        -258.17          0.00      1569.64   
11  1362928       41      0.00           0.00          0.00       420.42   
12  1363066       49     -0.15          -0.15          0.00        24.47   
13  1362746        1   -126.72        -126.72          0.00        22.59   
14  1363393       14      0.00           0.00          0.00        17.29   
15  1362964       98    -42.77         -42.77          0.00       609.31   
16  1362557       85      0.00           0.00          0.00       125.96   
17  1362897       93    -81.43         -81.43          0.00        81.42   
18  1363433       70    -10.00         -10.00          0.00      1913.92   
19  1362542        4     -0.15          -0.15          0.00      2483.62   
20  1363008       28   -128.23        -128.23          0.00      4602.30   
21  1363249       40    662.82         662.82          0.00      1399.37   
22  1362850       25    -78.82         -78.82          0.00      2424.44   
23  1363361       83    -22.44         -22.44          0.00        92.44   
24  1363107        9     -0.15          -0.15          0.00        24.43   
25  1363183       27      0.00           0.00          0.00      7375.04   
26  1363123       15   -200.00        -200.00          0.00      8297.49   
27  1362609       12   -817.25        -817.25          0.00      1164.74   
28  1362674       89    -10.00         -10.00          0.00       937.35   
29  1363170       57   -103.57        -103.57          0.00       103.57   

     tran_date     tran_time channel tran_code  
0    29.1.1995                            FK    
1     2.7.1995         53313      A       WD    
2    26.3.1995           834      A       IQ    
3    29.1.1995                    P       WD    
4     8.2.1995                    P       WD    
5    25.2.1995                    P       WD    
6     7.9.1995        160041      A       TR    
7    22.1.1995                    P       WD    
8    30.6.1995        235959              IN    
9    15.1.1995        115613              FU    
10    4.1.1995                    P       WD    
11    3.2.1995        154333      C       IQ    
12   23.2.1995                            FK    
13   23.8.1995                    P       WD    
14   10.1.1995         91047      C       IQ    
15  16.12.1995        155717      B       WD    
16   4.10.1995        230103      V       IQ    
17   26.8.1995        143001      V       TR    
18  30.11.1995        235959              FP    
19    8.2.1995                            FK    
20   17.5.1995                    P       WD    
21   14.1.1995        150516      A       DP    
22   6.11.1995                    P       WD    
23   22.1.1995                    P       WD    
24   10.1.1995                            FK    
25   15.1.1995         12518      A       IQ    
26   15.3.1995        155454      A       WD    
27    6.1.1995        115314      E       TR    
28   31.5.1995        235959              FP    
29   26.5.1995                    P       WD

In [105]:
df_checking_tran2 = pd.read_csv('df_checking_tran_clean.csv',',')
df_checking_tran2 = df_checking_tran2.drop(labels='Unnamed: 0', axis=1)
df_checking_tran2.head()

cust_id  tran_id  tran_amt  principal_amt  interest_amt  new_balance  \
0  1363481       26     -0.15          -0.15           0.0        58.99   
1  1362784       90   -200.00        -200.00           0.0      1380.69   
2  1363088       46      0.00           0.00           0.0       128.35   
3  1363306       32    -40.32         -40.32           0.0       433.06   
4  1363251       11    -87.88         -87.88           0.0      1919.55   

   tran_date tran_code  
0  29.1.1995      FK    
1   2.7.1995      WD    
2  26.3.1995      IQ    
3  29.1.1995      WD    
4   8.2.1995      WD

### df_credit_acct

In [7]:
df_credit_acct = pd.read_csv('twm_credit_acct.csv',';')
print(df_credit_acct.shape)
df_credit_acct.head(5)

(468, 9)


cust_id          acct_nbr  credit_limit  credit_rating account_active  \
0  1363160  4561143213631600          1000              0             Y    
1  1362487  4561143213624870          3000              0             Y    
2  1362548  4561143213625480          6700              0             Y    
3  1362752  4561143213627520          2400              0             N    
4  1363017  4561143213630170          1000              0             Y    

  acct_start_date acct_end_date  starting_balance  ending_balance  
0      12.10.1988           NaN            657.46          286.69  
1        8.8.1995           NaN              0.00         1548.23  
2      11.11.1986           NaN           6965.25           68.68  
3       9.10.1994      2.4.1995           2302.14            0.00  
4        8.2.1995           NaN              0.00         1000.00

In [112]:
df_credit_acct2 = pd.read_csv('df_credit_acct_clean.csv',',')
df_credit_acct2 = df_credit_acct2.drop(labels='Unnamed: 0', axis=1)
df_credit_acct2.head(5)

cust_id          acct_nbr  credit_limit  credit_rating account_active  \
0  1363160  4561143213631600          1000              0             Y    
1  1362487  4561143213624870          3000              0             Y    
2  1362548  4561143213625480          6700              0             Y    
3  1362752  4561143213627520          2400              0             N    
4  1363017  4561143213630170          1000              0             Y    

  acct_start_date  starting_balance  ending_balance  
0      12.10.1988            657.46          286.69  
1        8.8.1995              0.00         1548.23  
2      11.11.1986           6965.25           68.68  
3       9.10.1994           2302.14            0.00  
4        8.2.1995              0.00         1000.00

### df_credit_tran

In [73]:
df_credit_tran = pd.read_csv('twm_credit_tran.csv',';')
print(df_credit_tran.shape)
df_credit_tran.head(20)

(20167, 10)


cust_id  tran_id  tran_amt  principal_amt  interest_amt  new_balance  \
0   1363088       46   -121.49        -121.49          0.00      -141.00   
1   1363306       32   -220.64        -220.64          0.00      -520.13   
2   1362959       10      0.00           0.00          0.00     -3900.00   
3   1363429       13   -195.27        -195.27          0.00     -1739.74   
4   1363393       14    -37.32         -37.32          0.00      -198.98   
5   1363303       20   -180.00        -180.00          0.00      -180.00   
6   1363008       28   -102.28        -102.28          0.00      -740.23   
7   1362928       41    -25.14         -25.14          0.00      -843.42   
8   1363107        9    -72.06         -72.06          0.00       -72.06   
9   1362746        1    558.73         544.12         14.61      -492.99   
10  1363123       15   -247.64        -247.64          0.00     -2800.00   
11  1362611       19    611.44         604.28          7.16      -362.22   
12  1363387       11    -88.44         -88.44          0.00     -1000.00   
13  1362542        4     86.53          77.31          9.22      -577.29   
14  1362991       33    467.66         462.00          5.66      -342.34   
15  1363249       40      0.00           0.00          0.00     -2117.51   
16  1362905        2   -242.78        -242.78          0.00      -915.60   
17  1362609       12   -184.00        -184.00          0.00     -2631.72   
18  1363182       42   -271.36        -271.36          0.00     -3313.01   
19  1363356       46    230.15         224.59          5.56      -469.76   

     tran_date     tran_time channel tran_code  
0   20.10.1995        101144      E       CG    
1   13.10.1995         83115      E       CG    
2    26.2.1995         84129      K       IQ    
3     3.1.1995        191815      E       CG    
4     4.7.1995        100811      E       CG    
5     8.8.1995        203528      A       CA    
6    13.8.1995        122055      E       CG    
7    26.6.1995        200337      E       CG    
8    14.4.1995        181619      E       CG    
9     7.1.1995                    M       PM    
10   15.6.1995        221224      E       CG    
11    4.5.1995        142249      C       PM    
12    3.4.1995        113404      E       CG    
13    1.4.1995        214456      C       PM    
14    3.8.1995                    M       PM    
15   23.4.1995         91327      B       IQ    
16   29.1.1995        182723      E       CG    
17   28.2.1995        134637      E       CG    
18  12.12.1995          4416      E       CG    
19   3.11.1995                    M       PM

In [113]:
df_credit_tran2 = pd.read_csv('df_credit_tran_clean.csv',',')
df_credit_tran2 = df_credit_tran2.drop(labels='Unnamed: 0', axis=1)
df_credit_tran2.head(20)

cust_id  tran_id  tran_amt  principal_amt  interest_amt  new_balance  \
0   1363088       46   -121.49        -121.49          0.00      -141.00   
1   1363306       32   -220.64        -220.64          0.00      -520.13   
2   1362959       10      0.00           0.00          0.00     -3900.00   
3   1363429       13   -195.27        -195.27          0.00     -1739.74   
4   1363393       14    -37.32         -37.32          0.00      -198.98   
5   1363303       20   -180.00        -180.00          0.00      -180.00   
6   1363008       28   -102.28        -102.28          0.00      -740.23   
7   1362928       41    -25.14         -25.14          0.00      -843.42   
8   1363107        9    -72.06         -72.06          0.00       -72.06   
9   1362746        1    558.73         544.12         14.61      -492.99   
10  1363123       15   -247.64        -247.64          0.00     -2800.00   
11  1362611       19    611.44         604.28          7.16      -362.22   
12  1363387       11    -88.44         -88.44          0.00     -1000.00   
13  1362542        4     86.53          77.31          9.22      -577.29   
14  1362991       33    467.66         462.00          5.66      -342.34   
15  1363249       40      0.00           0.00          0.00     -2117.51   
16  1362905        2   -242.78        -242.78          0.00      -915.60   
17  1362609       12   -184.00        -184.00          0.00     -2631.72   
18  1363182       42   -271.36        -271.36          0.00     -3313.01   
19  1363356       46    230.15         224.59          5.56      -469.76   

     tran_date tran_code  
0   20.10.1995      CG    
1   13.10.1995      CG    
2    26.2.1995      IQ    
3     3.1.1995      CG    
4     4.7.1995      CG    
5     8.8.1995      CA    
6    13.8.1995      CG    
7    26.6.1995      CG    
8    14.4.1995      CG    
9     7.1.1995      PM    
10   15.6.1995      CG    
11    4.5.1995      PM    
12    3.4.1995      CG    
13    1.4.1995      PM    
14    3.8.1995      PM    
15   23.4.1995      IQ    
16   29.1.1995      CG    
17   28.2.1995      CG    
18  12.12.1995      CG    
19   3.11.1995      PM

In [71]:
df_credit_tran.loc[df_credit_tran['tran_time'] == 0].count()

cust_id          0
tran_id          0
tran_amt         0
principal_amt    0
interest_amt     0
new_balance      0
tran_date        0
tran_time        0
channel          0
tran_code        0
dtype: int64

### df_twm_customer

In [79]:
df_twm_customer = pd.read_csv('twm_customer.csv',';')
print(df_twm_customer.shape)
df_twm_customer.head(20)

(747, 15)


cust_id  income  age  years_with_bank  nbr_children gender  \
0   1362691   26150   46                5             1     M    
1   1362487    6605   71                1             0     M    
2   1363160   18548   38                8             0     F    
3   1362752   47668   54                3             0     F    
4   1362548   44554   59                9             2     F    
5   1362813   15684   53                3             2     F    
6   1363017       0   16                1             0     M    
7   1362609    1929   79                8             0     F    
8   1363486   39942   41                1             5     F    
9   1363078    9622   46                6             1     M    
10  1363282   25829   29                8             0     F    
11  1363343       0   15                0             0     M    
12  1362527   17622   44                1             0     M    
13  1362670    8877   26                5             0     F    
14  1362996   23351   51                1             2     F    
15  1362935       0   15                0             0     M    
16  1363465   10516   41                1             1     M    
17  1363404       0   17                2             0     M    
18  1362588    6354   66                6             0     M    
19  1363200   21015   18                3             0     M    

    marital_status name_prefix  \
0                2               
1                2               
2                1               
3                1               
4                4               
5                2               
6                1               
7                2               
8                4               
9                2    Mr.        
10               1    Ms.        
11               1               
12               2               
13               1               
14               2    Mrs.       
15               1               
16               4               
17               1               
18               2               
19               1               

                                           first_name  \
0   Donald                                        ...   
1   ChingDyi                                      ...   
2   Rosa                                          ...   
3   Lisa                                          ...   
4   Barbara                                       ...   
5   Terry                                         ...   
6   Tim                                           ...   
7   Sandy                                         ...   
8   Jill                                          ...   
9   Christopher                                   ...   
10  Betty                                         ...   
11  Robin                                         ...   
12  Chuck                                         ...   
13  Lynn                                          ...   
14  Robin                                         ...   
15  Duncan                                        ...   
16  Harvey                                        ...   
17  Jacob                                         ...   
18  Geneve                                        ...   
19  Owen                                          ...   

                                            last_name  street_nbr  \
0   Marek                                         ...        8298   
1   Moussavi                                      ...       10603   
2   Johnston                                      ...        8817   
3   Martin                                        ...         676   
4   O'Malley                                      ...        6578   
5   Eubanks                                       ...        6999   
6   McMillen                                      ...       10274   
7   Rayhanian                                     ...        6168   
8   Campbell                                      ...        3830   
9   McVicker  

In [53]:
df_twm_customer2 = df_twm_customer.copy()

In [85]:
df_twm_customer2.drop(labels=['name_prefix'], axis=1, inplace=True)
df_twm_customer2.head()

cust_id  income  age  years_with_bank  nbr_children gender  marital_status  \
0  1362691   26150   46                5             1     M                2   
1  1362487    6605   71                1             0     M                2   
2  1363160   18548   38                8             0     F                1   
3  1362752   47668   54                3             0     F                1   
4  1362548   44554   59                9             2     F                4   

                                          first_name  \
0  Donald                                        ...   
1  ChingDyi                                      ...   
2  Rosa                                          ...   
3  Lisa                                          ...   
4  Barbara                                       ...   

                                           last_name  street_nbr  \
0  Marek                                         ...        8298   
1  Moussavi                                      ...       10603   
2  Johnston                                      ...        8817   
3  Martin                                        ...         676   
4  O'Malley                                      ...        6578   

                                         street_name  postal_code  \
0  Second                                        ...        89194   
1  Daffodil                                      ...        90159   
2  Figueroa                                      ...        90024   
3  Humble                                        ...        90172   
4  C                                             ...        10138   

                                  city_name state_code  
0  Las Vegas                                      NV    
1  Los Angeles                                    CA    
2  Los Angeles                                    CA    
3  Los Angeles                                    CA    
4  New York City                                  NY

In [54]:
total = df_twm_customer2.isnull().sum().sort_values(ascending=False)
percent = (df_twm_customer2.isnull().sum()/df_twm_customer2.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

Total  Percent
state_code           0      0.0
city_name            0      0.0
postal_code          0      0.0
street_name          0      0.0
street_nbr           0      0.0
last_name            0      0.0
first_name           0      0.0
name_prefix          0      0.0
marital_status       0      0.0
gender               0      0.0
nbr_children         0      0.0
years_with_bank      0      0.0
age                  0      0.0
income               0      0.0
cust_id              0      0.0

In [86]:
df_twm_customer2.to_csv('df_twm_customer_cleaned.csv', sep=',', index=False)

#### df_savings_acct

In [46]:
df_savings_acct = pd.read_csv('twm_savings_acct.csv',';')
print(df_savings_acct.shape)
df_savings_acct.head(5)

(421, 9)


cust_id  acct_nbr  minimum_balance acct_type account_active  \
0  1363160  13631603              100      BS               Y    
1  1362487  13624873              500      MM               N    
2  1362548  13625483              100      BS               Y    
3  1362752  13627523              100      BS               Y    
4  1363017  13630173              100      BS               N    

  acct_start_date acct_end_date  starting_balance  ending_balance  
0        9.5.1988           NaN            113.04          122.54  
1       27.6.1994     25.8.1995            651.73           12.89  
2      11.11.1986           NaN            250.24          254.02  
3       20.2.1995           NaN            600.26          609.33  
4       27.4.1994    31.10.1995           2011.02         2901.35

In [47]:
df_savings_acct2 = df_savings_acct.copy()

In [48]:
total = df_savings_acct2.isnull().sum().sort_values(ascending=False)
percent = (df_savings_acct2.isnull().sum()/df_savings_acct2.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

Total   Percent
acct_end_date       375  0.890736
ending_balance        0  0.000000
starting_balance      0  0.000000
acct_start_date       0  0.000000
account_active        0  0.000000
acct_type             0  0.000000
minimum_balance       0  0.000000
acct_nbr              0  0.000000
cust_id               0  0.000000

In [49]:
to_drop = missing_data.head(1).index.tolist()
df_savings_acct2.drop(to_drop, axis=1, inplace=True)

In [50]:
df_savings_acct2.head()

cust_id  acct_nbr  minimum_balance acct_type account_active  \
0  1363160  13631603              100      BS               Y    
1  1362487  13624873              500      MM               N    
2  1362548  13625483              100      BS               Y    
3  1362752  13627523              100      BS               Y    
4  1363017  13630173              100      BS               N    

  acct_start_date  starting_balance  ending_balance  
0        9.5.1988            113.04          122.54  
1       27.6.1994            651.73           12.89  
2      11.11.1986            250.24          254.02  
3       20.2.1995            600.26          609.33  
4       27.4.1994           2011.02         2901.35

In [51]:
df_savings_acct2.to_csv('df_saving_acct_cleaned.csv', sep=',', index=False)

#### df_savings_tran

In [74]:
df_savings_tran = pd.read_csv('twm_savings_tran.csv',';')
print(df_savings_tran.shape)
df_savings_tran.head(30)

(11189, 10)


cust_id  tran_id  tran_amt  principal_amt  interest_amt  new_balance  \
0   1363481       26    136.03         136.03          0.00      1521.57   
1   1362746        1      2.83           0.00          2.83      2265.90   
2   1363251       11     13.56           0.00         13.56      5438.90   
3   1362542        4      0.76           0.00          0.76       610.92   
4   1363387       11    148.69         148.69          0.00       470.05   
5   1362609       12   -167.04        -167.04          0.00       360.76   
6   1362905        2     91.55          91.55          0.00       615.15   
7   1362926       18    271.95         271.95          0.00      1507.62   
8   1362968       10      0.89           0.00          0.89       355.23   
9   1362766       39   1894.54        1894.54          0.00     26016.69   
10  1363167       17      3.32           0.00          3.32      1333.27   
11  1362777        8    -35.78         -35.78          0.00        35.78   
12  1363128        9      2.75           0.00          2.75      2203.42   
13  1363038       21   -536.10        -536.10          0.00       825.09   
14  1362759       14      0.00           0.00          0.00      1398.00   
15  1362785        5      0.00           0.00          0.00        87.64   
16  1362698       36   -140.00        -140.00          0.00       321.80   
17  1363455       11      0.32           0.00          0.32       255.33   
18  1362939       25      3.30           0.00          3.30      1213.31   
19  1363452       40      2.34           0.00          2.34      1874.79   
20  1363353       24      1.61           0.00          1.61      1288.91   
21  1362945       20      4.50           0.00          4.50      1804.89   
22  1362665       22   -292.16        -292.16          0.00      2055.68   
23  1363454       25     74.07          74.07          0.00       781.28   
24  1363290       15    -40.00         -40.00          0.00      2517.92   
25  1363270        7      1.39           0.00          1.39       558.75   
26  1362875       17      0.00           0.00          0.00       447.98   
27  1362587       20    152.22         152.22          0.00      1487.18   
28  1363352        1    -20.00         -20.00          0.00      2019.40   
29  1363150       42    -40.00         -40.00          0.00       613.82   

     tran_date tran_time channel tran_code  
0   26.10.1995     84111      A       DP    
1    31.1.1995    235959              IN    
2    30.4.1995    235959              IN    
3    30.4.1995    235959              IN    
4    21.5.1995    144736      E       DP    
5     3.8.1995    213122      B       WD    
6    10.2.1995    132042      B       DP    
7    18.6.1995    234752      E       DP    
8    30.6.1995    235959              IN    
9    16.9.1995     83342      A       DP    
10   31.7.1995    235959              IN    
11    2.4.1995    181320      B       TR    
12   30.9.1995    235959              IN    
13  19.10.1995    202523      A       TR    
14    5.5.1995    131652      C       IQ    
15   28.3.1995    113130      H       IQ    
16   28.2.1995    154902      A       WD    
17  30.11.1995    235959              IN    
18   31.8.1995    235959              IN    
19  31.12.1995    235959              IN    
20   31.3.1995    235959              IN    
21   30.6.1995    235959              IN    
22   16.8.1995    142042      B       TR    
23  29.12.1995     20430      B       DP    
24   1.10.1995    220641      A       WD    
25   31.7.1995    235959              IN    
26  26.10.1995    193444      B       IQ    
27    1.8.1995    150624      E       DP    
28    2.1.1995    233954      A       WD    
29  15.11.1995     22355      A       WD

In [42]:
df_savings_tran2 = df_savings_tran.copy()

In [68]:
df_savings_tran2.loc[df_savings_tran2['tran_time'] == 0].count()

cust_id          0
tran_id          0
tran_amt         0
principal_amt    0
interest_amt     0
new_balance      0
tran_date        0
tran_time        0
channel          0
tran_code        0
dtype: int64

In [83]:
df_savings_tran2.drop(labels=['tran_time', 'channel'], axis=1, inplace=True)
df_savings_tran2.head()

cust_id  tran_id  tran_amt  principal_amt  interest_amt  new_balance  \
0  1363481       26    136.03         136.03          0.00      1521.57   
1  1362746        1      2.83           0.00          2.83      2265.90   
2  1363251       11     13.56           0.00         13.56      5438.90   
3  1362542        4      0.76           0.00          0.76       610.92   
4  1363387       11    148.69         148.69          0.00       470.05   

    tran_date tran_code  
0  26.10.1995      DP    
1   31.1.1995      IN    
2   30.4.1995      IN    
3   30.4.1995      IN    
4   21.5.1995      DP

In [43]:
total = df_savings_tran2.isnull().sum().sort_values(ascending=False)
percent = (df_savings_tran2.isnull().sum()/df_savings_tran2.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

Total  Percent
tran_code          0      0.0
channel            0      0.0
tran_time          0      0.0
tran_date          0      0.0
new_balance        0      0.0
interest_amt       0      0.0
principal_amt      0      0.0
tran_amt           0      0.0
tran_id            0      0.0
cust_id            0      0.0

In [84]:
df_savings_tran2.to_csv('df_saving_tran_cleaned.csv', sep=',', index=False)

#### df_transactions

In [117]:
df_transactions = pd.read_csv('twm_transactions.csv',';')
print(df_transactions.shape)
df_transactions.tail(20)

(77580, 10)


tran_id          acct_nbr  tran_amt  principal_amt  interest_amt  \
77560       42  4561143213628770      0.00           0.00          0.00   
77561       23          13629042   -144.20        -144.20          0.00   
77562       22          13631263      4.40           0.00          4.40   
77563        7          13633992   -184.82        -184.82          0.00   
77564       44          13630732    173.92         173.92          0.00   
77565       16          13629323      1.77           0.00          1.77   
77566        4          13633462      0.00           0.00          0.00   
77567       32          13626192    -10.00         -10.00          0.00   
77568       20  4561143213632960   -168.65        -168.65          0.00   
77569       19          13629122      0.00           0.00          0.00   
77570        7          13629782   -506.78        -506.78          0.00   
77571       27          13632092     -0.15          -0.15          0.00   
77572       31          13632952      0.00           0.00          0.00   
77573       95          13625522   -104.49        -104.49          0.00   
77574        1  4561143213629730   -223.35        -223.35          0.00   
77575       77          13631752    -95.71         -95.71          0.00   
77576       94          13626772    -93.90         -93.90          0.00   
77577       42          13625262    -10.35         -10.35          0.00   
77578       98          13627052   -423.80        -423.80          0.00   
77579       57  4561143213632570      0.00           0.00          0.00   

       new_balance   tran_date  tran_time channel tran_code  
77560       -80.00  13.12.1995     132859      A       IQ    
77561       326.74   18.3.1995       1854      A       TR    
77562      3237.66   30.9.1995     235959              IN    
77563       527.54  21.11.1995          0      P       WD    
77564       187.94   13.1.1995     103908      B       DP    
77565       708.37   31.5.1995     235959              IN    
77566      3215.50   30.1.1995     115948      A       IQ    
77567      1570.80   31.3.1995     235959              FP    
77568     -1191.55    6.3.1995     231040      E       CG    
77569        19.82   13.1.1995     161639      A       IQ    
77570      5288.69    3.1.1995     163754      E       WD    
77571      1168.75   21.2.1995          0              FK    
77572         0.00   12.4.1995     112550      A       IQ    
77573       320.07   17.7.1995          0      P       WD    
77574      -223.35    4.2.1995      25421      E       CG    
77575        95.71   14.2.1995     161001      A       TR    
77576       824.36   11.3.1995          0      P       WD    
77577        10.36   26.8.1995          0      P       WD    
77578       162.28   13.6.1995     110209      E       WD    
77579     -2420.91  18.10.1995     104027      A       IQ

In [38]:
df_transactions2 = df_transactions.copy()

In [67]:
df_transactions2.loc[df_transactions2['tran_time'] == 0].count()

tran_id          23564
acct_nbr         23564
tran_amt         23564
principal_amt    23564
interest_amt     23564
new_balance      23564
tran_date        23564
tran_time        23564
channel          23564
tran_code        23564
dtype: int64

In [80]:
df_transactions2.drop(labels=['tran_time', 'channel'], axis=1, inplace=True)

In [81]:
df_transactions2.head()

tran_id  acct_nbr  tran_amt  principal_amt  interest_amt  new_balance  \
0       27  13625623      0.00           0.00           0.0      3753.34   
1       97  13628392      0.00           0.00           0.0       254.49   
2       21  13630842    -97.57         -97.57           0.0      3819.56   
3       44  13631412     -0.15          -0.15           0.0       224.05   
4       31  13625722      0.00           0.00           0.0       240.55   

    tran_date tran_code  
0  21.10.1995      IQ    
1    5.2.1995      IQ    
2   23.7.1995      WD    
3   30.1.1995      FK    
4   25.1.1995      IQ

In [39]:
total = df_transactions2.isnull().sum().sort_values(ascending=False)
percent = (df_transactions2.isnull().sum()/df_transactions2.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

Total  Percent
tran_code          0      0.0
channel            0      0.0
tran_time          0      0.0
tran_date          0      0.0
new_balance        0      0.0
interest_amt       0      0.0
principal_amt      0      0.0
tran_amt           0      0.0
acct_nbr           0      0.0
tran_id            0      0.0

In [82]:
df_transactions2.to_csv('df_transactions_cleaned.csv', sep=',', index=False)

# Create Database

In [108]:
db_connection = create_engine('sqlite:///mini3.db')
db_con = db_connection.connect()
df_transactions2.to_sql('all_trans', db_con, if_exists='replace', index=False)
df_savings_tran2.to_sql('savings_trans', db_con, if_exists='replace', index=False)
df_savings_acct2.to_sql('savings_acct', db_con, if_exists='replace', index=False)
df_twm_customer2.to_sql('customer_info', db_con, if_exists='replace', index=False)
df_credit_tran2.to_sql('credit_trans', db_con, if_exists='replace', index=False)
df_credit_acct2.to_sql('credit_acct', db_con, if_exists='replace', index=False)
df_checking_tran2.to_sql('checking_trans', db_con, if_exists='replace', index=False)
df_checking_acct2.to_sql('checking_acct', db_con, if_exists='replace', index=False)
df_accounts2.to_sql('all_accts', db_con, if_exists='replace', index=False)
